# Overview

## 1. Ask Interesting Questions
- Ensure questions are specific, measurable, achievable, relevant, and time-bound (SMART).
## 2. Get the Data
## 3. Preprocessing the Data
## 4. Explore the Data
## 5. Model the Data
## 6. Communicate and Visualize the Results / Is the Question answered? Why?
- Present insights clearly and concisely.
- Use visualizations to support findings.
- Document key insights and actionable recommendations.
- Prepare a 2-page management summary.
## 7. Manage and Document the Project
- Ensure all Jupyter notebook cells are executed before submission.
- Explain all steps, choices, and reasoning.
- Document dataset selection, preprocessing, and analysis challenges.
- Include descriptions of data sources and installation instructions in an `install_data.txt` file if required.
- Submit a ZIP file with the notebook, required data, and documentation.
## 8. Collaboration and Workflow
- Document contributions of each member.
## 9. Learning and Reflection
- Reflect on new tools and techniques learned.
- Evaluate difficulties faced during analysis.
- Document future improvements and considerations.


## 1. Ask Interesting Questions


#### Our 4 Starting Question we decided to change to a single one:

1. How well can the level of corruption in a European country be quantified?

This questions is too broad and may lack specificity. Also not a good question to answer within a notebook.
The definition of "quantified" needs clarification—are we looking at an index, a model, or a metric?

2. Are there different forms of corruption prevalent in different European countries?

While interesting, it may be challenging to obtain granular, comparable data across countries.
Requires deep qualitative insights, which may not be fully captured by available datasets.

3. What characteristics of a country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

4. What characteristics of a country predict an increase or decrease in the level of corruption?

Why it's promising:

Focuses on change over time, enabling trend analysis.
Useful for policy evaluation and forecasting.
Encourages a deeper exploration of temporal datasets (e.g., economic reforms, governance improvements, etc.).
Can help identify early warning signs for rising corruption.
Potential Challenges:

Requires time-series data and careful handling of lag effects.
External factors (global economic crises, political events) may introduce noise.


The following question seems the most promising and we decided to only choose this one:

# What characteristics of a european country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

maybe we could tackle this one but lets find out later: Which country characteristics (e.g., economic, political, social indicators) best predict the level of corruption in European countries, and how do these characteristics relate to any gap between actual and perceived corruption?



## 2. Get the Data

As we have decided to focus on the continent of europe the first challenge will be to create a dataset of all european countries. To make this notebook not substantially long we will provide the finished data **europe_countries** to you. It shows all european countries and their ISO3 Code and ISO2 Code which should allow better preprocessing later.

In [224]:
import numpy as np
import pandas as pd
import os
import glob
import kagglehub


In [225]:
countries = pd.read_csv("../data/processed/europe_countries.csv")

In [226]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    49 non-null     object
 1   ISO3 Code  49 non-null     object
 2   ISO2 Code  49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [227]:
countries['Country'].unique()

array(['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium',
       'Bosnia & Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
       'Italy', 'Kosovo2', 'Kosovo', 'Latvia', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco',
       'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland',
       'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey',
       'Ukraine', 'United Kingdom', 'Vatican City'], dtype=object)

When looking at european countrie numbers, many figures fly arround, our data includes 48 countries, and of course the most promising ones. As we will see later most of the other datasets aint serve data for all countries anyway. Especially for small countries like Vatican or Monaco, etc.

Why the Corruption Perceptions Index (CPI) Dataset is a Strong Core for Our Project

The Corruption Perceptions Index (CPI) from Transparency International serves as an excellent core dataset for analyzing the factors influencing corruption in European countries. Below are the key reasons why this dataset is well-suited for our project:

Comprehensive and Reliable Data Source:
Credibility: Transparency International is a globally recognized authority on corruption-related data, ensuring the dataset's legitimacy and accuracy.
Standardized Methodology: The CPI is calculated using multiple expert assessments and business surveys, making it a robust and widely accepted measure of perceived corruption.
Consistency: The data is available annually from 2012 to 2023, providing a solid historical basis for trend analysis and predictions.

### Corruption Perceptions Index (CPI) from Transparency International.
Data Set that shows preceived corruption of countries and rank them.

Link to data: https://images.transparencycdn.org/images/CPI2023_FullDataSet.zip
https://www.transparency.org/en/news/how-cpi-scores-are-calculated

 0   Economy ISO3                                
 1   Economy Name                                 
 2   Year                                           
 3   Corruption Perceptions Index Rank            
 4   Corruption Perceptions Index Score           
 5   Corruption Perceptions Index Sources        
 6   Corruption Perceptions Index Standard Error

We took the xlsx and transformed it to this more comapct version with only features we need and only european countries. Data points get explained at the end of each processing.

In [228]:
cpi_data = pd.read_csv("../data/processed/CPI.csv")

In [231]:
iso3_europe_full = set(countries["ISO3 Code"])

iso3_europe_cpi = set(cpi_data["Economy ISO3"])

print('Lenght of all europe countries in our list: ',len(iso3_europe_full))
print('Lenght of cpi countriest:', len(iso3_europe_cpi))

print('Countries mssing')

missing_countries = iso3_europe_full - iso3_europe_cpi
print('Countries missing:', missing_countries)



Lenght of all europe countries in our list:  49
Lenght of cpi countriest: 42
Countries mssing
Countries missing: {'AND', 'SMR', 'RKS', 'VAT', 'XKX', 'MCO', 'LIE'}


Countries missing: Andora, Liechtenstein, Kosovo, San Marino, Vatikan, Gibraltar, Azerbaijan. We could neclect this countries due to their small size and impact.

As they are missing in our main dataset lets exclude them from the countries list

In [235]:
# Filter out missing countries from the 'countries' DataFrame
countries = countries[~countries["ISO3 Code"].isin(missing_countries)]

# Save the filtered DataFrame if needed
countries.to_csv("../data/processed/europe_countries_filtered.csv", index=False)

In [236]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 0 to 47
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    42 non-null     object
 1   ISO3 Code  42 non-null     object
 2   ISO2 Code  42 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [237]:
cpi_data['Year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023])

In [238]:
cpi_data.rename(columns={'Economy ISO3': 'ISO3 Code'}, inplace=True)

In [239]:
cpi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   ISO3 Code                                    504 non-null    object 
 1   Economy Name                                 504 non-null    object 
 2   Year                                         504 non-null    int64  
 3   Corruption Perceptions Index Rank            491 non-null    float64
 4   Corruption Perceptions Index Score           504 non-null    float64
 5   Corruption Perceptions Index Sources         504 non-null    float64
 6   Corruption Perceptions Index Standard Error  504 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 27.7+ KB


#### CPI Dataset Summary
* Corruption Perceptions Index Rank (float64):
This represents the ranking of the country based on its Corruption Perceptions Index (CPI) score, with rank 1 being the least corrupt country.
Lower ranks indicate better perceived transparency.

* Corruption Perceptions Index Score (float64):
This score quantifies perceived corruption levels on a scale of 0 to 100, where:
0 = highly corrupt
100 = very clean (low corruption perception)
It is calculated using multiple data sources, standardized, and aggregated.

* Corruption Perceptions Index Sources (float64):
The number of sources used to calculate the CPI score for a given country.
The CPI uses multiple expert assessments and business surveys; having more sources increases reliability.

*  Corruption Perceptions Index Standard Error (float64):
This represents the uncertainty or variability in the CPI score, showing how much variation exists among different data sources.
A lower standard error indicates a more reliable score, while a higher one suggests more disagreement or variability in corruption perceptions across sources.

### GDP Data

We decided to go for GDP per Capita (PPP - Purchasing Power Parity)

Why?

Adjusted for cost of living, making it a fairer comparison across countries.
Helps analyze the standard of living and economic well-being in relation to corruption.
Frequently used in corruption-related studies to measure economic development.

Definition: A country's gross domestic product (GDP) at purchasing power parity (PPP) per capita is the PPP value of all final goods and services produced within an economy in a given year, divided by the average (or mid-year) population for the same year. This is similar to nominal GDP per capita but adjusted for the cost of living in each country.



### Processignt the World Bank Indicators and form one single wpi dataset

### Function to filter the datasets on our european country list.

In [240]:
def get_countries(df, countries_df, iso="ISO3 Code"):
    try:
        
        df.rename(columns={'Country Code': 'ISO3 Code'}, inplace=True)

        
        if iso not in countries_df.columns:
            raise KeyError(f"Column '{iso}' not found in countries DataFrame")
        
        # Process the ISO codes and clean whitespace
        iso_europe = set(countries_df[iso].dropna().astype(str).str.strip())
        iso_data = set(df[iso].dropna().astype(str).str.strip())
        
        # Print missing countries for debugging
        missing_countries = iso_data - iso_europe
        if missing_countries:
            print("Missing countries:", missing_countries)
        
        # Filter the DataFrame based on valid ISO codes
        filtered_df = df[df[iso].isin(iso_europe)]
        
        return filtered_df
    
    except KeyError as err:
        print(f"KeyError: {err}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


### Conversion of the raw csv to a comprimised Data Frame

In [241]:
def convert_worldbank_df(df, value_name):
    
    df =df.drop(labels=["Series Code", "Series Name"], axis= 1)

    #df = df.drop(df.tail(8).index)

    df.columns = df.columns.str.replace(r"\[.*?\]", "", regex=True).str.strip()
    
    df = df.melt(
    id_vars=['Country Name', 'ISO3 Code'],
    var_name='Year',
    value_name=value_name
)
    
    df.reset_index(drop=True, inplace=True)

    
    return df

### Loop to create a unified dataset

So as the process would be the same for every dataset of the worldbank we created a loop and combined the end results to have a better overview in the code. You will find a overall description of every data point after the code.

In [242]:
# Define your datasets info
datasets_info = {
    "rule_of_law": {
        "file_path": "rule_of_law",
        "value_name": "Rule_of_law"
    },
    "government_effectiveness": {
        "file_path": "government_effectiveness",
        "value_name": "Gov_effectiveness"
    },
    "control_of_corruption": {
        "file_path": "control_of_corruption",
        "value_name": "Control_of_corruption"
    },
    "fdi": {
        "file_path": "fdi",
        "value_name": "Fdi"
    },
    "gdp_per_capita": {
        "file_path": "gdp_per_capita",
        "value_name": "Gdp_per_capita"
    },
    "gini": {
        "file_path": "gini",
        "value_name": "Gini"
    },
    "unemployement": {
        "file_path": "unemployement",
        "value_name": "Unemployement"
    },
      "political_stability": {
        "file_path": "political_stability",
        "value_name": "Political_stability"
    },
}


In [243]:
def process_dataset(file_path, value_name, countries_df=countries):
    """
    Reads a CSV file and processes it into a standardized format:
      - Reads the CSV
      - Converts it using 'convert_worldbank_df' (user-defined)
      - Matches and filters countries using 'get_countries' (user-defined)
      - Keeps only [Country Name, Country Code, Year, <value_name>]

    Parameters:
        file_path (str): Path to the CSV file (without the .csv extension).
        value_name (str): Name to be assigned to the indicator value column.
        countries_df (pd.DataFrame): A DataFrame containing valid country information
                                     for filtering/mapping.

    Returns:
        pd.DataFrame: Processed DataFrame with columns:
                      ['Country Name', 'Country Code', 'Year', <value_name>]
    """
    full_path = f'../data/raw/{file_path}.csv'
    
    # Check if file exists
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"File not found: {full_path}")
    
    # Read CSV
    df = pd.read_csv(full_path)
    
    df.rename(columns={'Country Code': 'ISO3 Code'}, inplace=True)

        
    # Convert the World Bank style data to a long format with 'Year' and 'value_name'
    df = convert_worldbank_df(df=df, value_name=value_name)
    
    # Filter/match country codes and names with european country list
    df = get_countries(df=df, countries_df=countries_df)


    df = df[['ISO3 Code', 'Year', value_name]]
    
    print(df.shape)
    
    return df

In [244]:
def combine_datasets(datasets_info, countries_df):
    """
    Iterates over all datasets in datasets_info, processes each one,
    and merges them into a single DataFrame on ['Country Code', 'Year'].

    Parameters:
        datasets_info (dict): Dictionary containing dataset info with keys like:
                              {
                                  "rule_of_law": {
                                      "file_path": "rule_of_law",
                                      "value_name": "Rule_of_law"
                                  },
                                  ... 
                              }
        countries_df (pd.DataFrame): A DataFrame containing valid country information.

    Returns:
        pd.DataFrame: Final merged DataFrame with columns:
                      ['Country Code', 'Year', <all value_names>]
    """
    merged_df = None

    for ds_name, info in datasets_info.items():
        file_path = info["file_path"]
        value_name = info["value_name"]
        
        try:
            # Process each dataset
            temp_df = process_dataset(
                file_path=file_path,
                value_name=value_name,
            )
            print(f"Successfully processed: {ds_name}")

            # Ensure only necessary columns are present
            temp_df = temp_df[['ISO3 Code', 'Year', value_name]]
            
            # Merge into our final DF
            if merged_df is None:
                merged_df = temp_df
            else:
                merged_df = pd.merge(
                    merged_df,
                    temp_df,
                    on=['ISO3 Code', 'Year'],
                    how='outer'
                )

        except Exception as e:
            print(f"Error processing {ds_name}: {e}")
            continue

    if merged_df is not None:
        # Merge with the reference countries DataFrame to ensure all countries are included
        merged_df = pd.merge(countries_df[['ISO3 Code']], merged_df, on='ISO3 Code', how='left')
        merged_df.sort_values(by=['ISO3 Code', 'Year'], inplace=True)
        merged_df.reset_index(drop=True, inplace=True)

    return merged_df


In [245]:
# Combine them all
world_bank = combine_datasets(datasets_info, countries_df=countries)

# Inspect the merged DataFrame
print(world_bank.head())
print(world_bank.columns)

Missing countries: {'GRL', 'SMR', 'AND', 'TJK', 'LIE', 'KAZ', 'XKX', 'UZB', 'KGZ', 'TKM', 'ARM', 'AZE', 'MCO', 'JEY'}
(504, 3)
Successfully processed: rule_of_law
Missing countries: {'GRL', 'SMR', 'AND', 'TJK', 'LIE', 'KAZ', 'XKX', 'UZB', 'KGZ', 'TKM', 'ARM', 'AZE', 'MCO', 'JEY'}
(504, 3)
Successfully processed: government_effectiveness
Missing countries: {'GRL', 'SMR', 'AND', 'TJK', 'LIE', 'KAZ', 'XKX', 'UZB', 'KGZ', 'TKM', 'ARM', 'AZE', 'MCO', 'JEY'}
(504, 3)
Successfully processed: control_of_corruption
Missing countries: {'GRL', 'SMR', 'CHI', 'AND', 'TJK', 'KAZ', 'XKX', 'UZB', 'KGZ', 'FRO', 'GIB', 'TKM', 'ARM', 'AZE', 'MCO', 'IMN', 'LIE'}
(504, 3)
Successfully processed: fdi
Missing countries: {'AND', 'SMR', 'XKX', 'GIB', 'AZE', '(Europe)', 'MCO', 'LIE'}
(504, 3)
Successfully processed: gdp_per_capita
Missing countries: {'EGY', 'ISR', 'QAT', 'NCL', 'RWA', 'SGP', 'PSE', 'URY', 'IDN', 'MUS', 'BHR', 'JAM', 'LKA', 'PLW', 'NPL', 'PYF', 'ARG', 'KIR', 'DMA', 'LCA', 'ZMB', 'KGZ', 'SOM', 'G

## Worldbank Data Indicators

### 1. Basic Information

- **Data Source**: [https://databank.worldbank.org/indicator/NY.GDP.PCAP.CD/1ff4a498/Popular-Indicators?l=en#advancedDownloadOptions](https://databank.worldbank.org/indicator/NY.GDP.PCAP.CD/1ff4a498/Popular-Indicators?l=en#advancedDownloadOptions)
- **Description**: The dataset contains various economic indicators such as GDP per capita, foreign direct investment (FDI), unemployment rates, and governance indicators for European countries from 2012 to 2023.
- **Year/Coverage**: 2012-2023

### 2. Key Variables / Columns

| **Column Name**         | **Type** | **Description**                                                                          |
| ----------------------- | -------- | ---------------------------------------------------------------------------------------- |
| ISO3 Code               | object   | ISO 3 code of the country                                                                |
| Year                    | object   | Year of the recorded value                                                               |
| Rule\_of\_law           | float64  | Measures confidence in legal systems and contract enforcement (range: -2.5 to 2.5).      |
| Gov\_effectiveness      | float64  | Measures the quality of public services and policy implementation (range: -2.5 to 2.5).  |
| Control\_of\_corruption | float64  | Measures perceptions of corruption in public power (range: -2.5 to 2.5).                 |
| Fdi                     | float64  | Foreign direct investment, net inflows as a percentage of GDP.                           |
| Gdp\_per\_capita        | float64  | GDP per capita adjusted for purchasing power parity (PPP).                               |
| Gini                    | float64  | Gini index measuring income inequality (0 = perfect equality, 100 = perfect inequality). |
| Unemployement           | float64  | Total unemployment as a percentage of the total labor force.                             |

### 3. Data Cleaning / Transformation

- **Original Format**: Wide format with years as columns.
- **Filtering**: Retained only European countries.
- **Columns Kept/Removed**: Dropped unnecessary columns like `Series Code` and `Series Name`.
- **Data Type Conversions**: Converted `Year` to string and numeric columns to `float64`.
- **Handling Missing Values**: Missing values are left as NaN to indicate data gaps.

### 4. Data Context & Usage

#### GDP per Capita (PPP - Purchasing Power Parity)

- **Definition**: GDP at purchasing power parity (PPP) per capita is the PPP value of all final goods and services produced within an economy in a given year, divided by the mid-year population.
- **Range**: Measured in international dollars, adjusted for cost of living.
- **Note**: Helps compare living standards across countries.

#### Foreign Direct Investment (FDI), net inflows (% of GDP)

- **Definition**: Measures net inflows of investment to acquire a lasting management interest in an enterprise operating in another economy.
- **Range**: Expressed as a percentage of GDP.
- **Note**: Includes equity capital, reinvestment of earnings, and short/long-term capital.
- **Long definition**: Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows net inflows (new investment inflows less disinvestment) in the reporting economy from foreign investors, and is divided by GDP.
Source: International Monetary Fund, International Financial Statistics and Balance of Payments databases, World Bank, International Debt Statistics, and World Bank and OECD GDP estimates.

#### Unemployment Rate (% of total labor force)

- **Definition**: Refers to the share of the labor force that is without work but available for and seeking employment.
- **Range**: Expressed as a percentage.
- **Note**: Definitions of labor force and unemployment differ by country.

#### Gini Index

- **Definition**: Measures the extent to which the distribution of income among individuals deviates from a perfectly equal distribution.
- **Range**: 0 represents perfect equality; 100 implies perfect inequality.
- **Note**: A higher Gini index indicates greater inequality.
- **Long definition**: Gini index measures the extent to which the distribution of income (or, in some cases, consumption expenditure) among individuals or households within an economy deviates from a perfectly equal distribution. A Lorenz curve plots the cumulative percentages of total income received against the cumulative number of recipients, starting with the poorest individual or household. The Gini index measures the area between the Lorenz curve and a hypothetical line of absolute equality, expressed as a percentage of the maximum area under the line.

#### Rule of Law

- **Definition**: Captures perceptions of confidence in the legal system, contract enforcement, and likelihood of crime and violence.
- **Range**: -2.5 to 2.5.
- **Note**: Higher values indicate stronger rule of law perceptions.
- **Long definition**: Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society, and in particular the quality of contract enforcement, property rights, the police, and the courts, as well as the likelihood of crime and violence. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution

#### Control of Corruption

- **Definition**: Measures perceptions of how public power is exercised for private gain.
- **Range**: -2.5 to 2.5.
- **Note**: Higher values indicate better control of corruption.
- **Long definition**: Control of Corruption captures perceptions of the extent to which public power is exercised for private gain, including both petty and grand forms of corruption, as well as "capture" of the state by elites and private interests. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution.

#### Government Effectiveness

- **Definition**: Measures perceptions of the quality of public services and policy implementation.
- **Range**: -2.5 to 2.5.
- **Note**: Higher values indicate more effective governance.
- **Long definition**: Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy formulation and implementation, and the credibility of the government's commitment to such policies.
Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e.

#### Political Stability and Absence of Violence/Terrorism: Estimate
- **Definition**: Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence, including terrorism.
- **Range**: -2.5 to 2.5. in units of a standard normal distribution,
- **Note**: Higher values indicate more effective governance.

### World Happiness Report

Adding data from the World Happiness Report (WHR) to our corruption analysis gives us a fresh perspective that goes beyond just looking at economic and governance factors. While datasets like the World Bank’s Control of Corruption Index (CCI) and the Corruption Perceptions Index (CPI) mainly focus on how institutions handle corruption, the WHR brings in social and well-being aspects. This helps us get a more complete picture of how corruption affects people's daily lives and what social factors might be linked to it. Looking at things like happiness, social support, and life satisfaction can give us valuable insights into the broader impacts of corruption and how it might be tackled more effectively.

In [247]:
happiness = pd.read_excel("../data/raw/world_happiness_report.xls")

In [248]:
from fuzzywuzzy import process

def apply_iso3_code(df, countries_df, country_column_df):
    """
    Merges 'df' (with country names in 'country_column_df') against 'countries_df' 
    (with 'Country' and 'ISO3 Code') to add the 'ISO3 Code' column to 'df'.
    Uses fuzzy matching to resolve discrepancies in country names.

    Parameters:
        df (pd.DataFrame): Your original DataFrame with country names.
        countries_df (pd.DataFrame): DataFrame with at least two columns:
                                     - 'Country': textual name of the country
                                     - 'ISO3 Code': ISO3 code (e.g. FRA, DEU, etc.)
        country_column_df (str): Column name in 'df' that contains country names.

    Returns:
        pd.DataFrame: Updated DataFrame that includes the 'ISO3 Code'.
    """

    # Step 1: Print unique country names for debugging
    print("Country names in df:", df[country_column_df].unique())
    print("Country names in countries_df:", countries_df['Country'].unique())

    # Step 2: Apply fuzzy matching to standardize country names
    def match_country_name(country, country_list):
        match, score = process.extractOne(country, country_list)
        if score >= 95:  # Confidence threshold for fuzzy matching
            print(f"Matched: {country} -> {match} (Score: {score})")
            return match
        else:
            return country  # Keep the original name if no good match found

    df[country_column_df] = df[country_column_df].apply(lambda x: match_country_name(x, countries_df['Country'].tolist()))

    # Step 3: Merge on the country name after standardization
    merged = df.merge(
        countries_df[['Country', 'ISO3 Code']],
        how='left',
        left_on=country_column_df,
        right_on='Country'
    )

    # Step 4: Remove rows where ISO3 Code could not be assigned (optional)
    merged = merged[~merged['ISO3 Code'].isna()]

    # Step 5: Drop the redundant 'Country' column after merging
    merged.drop(columns=['Country'], inplace=True)

    return merged


In [249]:
happiness = apply_iso3_code(df=happiness, countries_df=countries, country_column_df="Country name")

Country names in df: ['Afghanistan' 'Albania' 'Algeria' 'Angola' 'Argentina' 'Armenia'
 'Australia' 'Austria' 'Azerbaijan' 'Bahrain' 'Bangladesh' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cambodia'
 'Cameroon' 'Canada' 'Central African Republic' 'Chad' 'Chile' 'China'
 'Colombia' 'Comoros' 'Congo (Brazzaville)' 'Congo (Kinshasa)'
 'Costa Rica' 'Croatia' 'Cuba' 'Cyprus' 'Czechia' 'Denmark' 'Djibouti'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Estonia' 'Eswatini'
 'Ethiopia' 'Finland' 'France' 'Gabon' 'Gambia' 'Georgia' 'Germany'
 'Ghana' 'Greece' 'Guatemala' 'Guinea' 'Guyana' 'Haiti' 'Honduras'
 'Hong Kong S.A.R. of China' 'Hungary' 'Iceland' 'India' 'Indonesia'
 'Iran' 'Iraq' 'Ireland' 'Israel' 'Italy' 'Ivory Coast' 'Jamaica' 'Japan'
 'Jordan' 'Kazakhstan' 'Kenya' 'Kosovo' 'Kuwait' 'Kyrgyzstan' 'Laos'
 'Latvia' 'Lebanon' 'Lesotho' 'Liberia' 'Libya' 'Lithuania' 'Luxembourg'
 'Ma

In [250]:
happiness.rename(columns={'year': 'Year'}, inplace=True)

In [251]:
happiness = happiness.sort_values(by=["Country name", "Year"])
happiness.reset_index(drop=True, inplace=True)
happiness = happiness[happiness['Year'] >= 2012]

In [252]:
happiness = happiness.drop(axis=1, labels=['Perceptions of corruption', 'Log GDP per capita'])

Getting the Report From World Happiness Report: https://worldhappiness.report/data/

In [253]:
happiness.head()

,Country name,Year,Life Ladder,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Positive affect,Negative affect,ISO3 Code
4,Albania,2012,5.510124,0.784502,68.160004,0.601512,-0.172988,0.553473,0.271393,ALB
5,Albania,2013,4.550648,0.759477,68.440002,0.631830,-0.131339,0.540751,0.338379,ALB
6,Albania,2014,4.813763,0.625587,68.720001,0.734648,-0.028801,0.572945,0.334543,ALB
7,Albania,2015,4.606651,0.639356,69.000000,0.703851,-0.084982,0.579072,0.350427,ALB
8,Albania,2016,4.511101,0.638411,69.025002,0.729819,-0.021136,0.567080,0.321706,ALB



- **Life Ladder**  
  - **Description:** A measure of subjective well-being based on the Cantril ladder, where respondents rate their life satisfaction on a scale from 0 to 10.  
  - **Technical Methodology:** Survey respondents rate their current life relative to their “best possible” and “worst possible” life. The national average is computed from individual responses.  
  - **Range:** 0 to 10.

- **Social support**  
  - **Description:** Perceived availability of social connections and support from friends and family during difficult times.  
  - **Technical Methodology:** Based on the percentage of respondents who report having someone to rely on in times of need.  
  - **Range:** 0 to 1 (proportion of respondents).

- **Healthy life expectancy at birth**  
  - **Description:** The average number of years a newborn is expected to live in good health.  
  - **Technical Methodology:** Derived from the World Health Organization (WHO) life tables and estimates. It accounts for mortality rates and morbidity data.  
  - **Range:** Typically between 50 to 85 years.

- **Freedom to make life choices**  
  - **Description:** The perception of personal freedom to make life choices.  
  - **Technical Methodology:** Based on survey questions where respondents report their level of freedom in decision-making. The national average is calculated.  
  - **Range:** 0 to 1 (proportion of positive responses).

- **Generosity**  
  - **Description:** The level of charitable behavior and generosity among the population.  
  - **Technical Methodology:** Based on self-reported recent donations and willingness to help strangers. The score is adjusted for income differences across countries.  
  - **Range:** Typically between -0.2 to 0.8 (due to standardization).

- **Positive affect**  
  - **Description:** The frequency of experiencing positive emotions such as laughter, joy, and enjoyment.  
  - **Technical Methodology:** Measured through survey questions where respondents report their experiences of positive emotions over the previous day.  
  - **Range:** 0 to 1 (proportion of positive responses).

- **Negative affect**  
  - **Description:** The frequency of experiencing negative emotions such as worry, sadness, and anger.  
  - **Technical Methodology:** Measured through survey questions where respondents report their experiences of negative emotions over the previous day.  
  - **Range:** 0 to 1 (proportion of negative responses).

In [200]:
### UN Data 

Source : https://dataunodc.un.org/dp-crime-corruption-offences <br>
Variables : https://dataunodc.un.org/sites/dataunodc.un.org/files/metadata_corruption_and_economic_crime.pdf <br>
Measurement unit: Rate per 100,000 population (R𝑎𝑡𝑒 = 100,000∗ 𝑂𝑓𝑓𝑒𝑛𝑐𝑒𝑠/ 𝑃𝑜𝑝𝑢𝑙𝑎𝑡𝑖𝑜n)<br>
Countries: Used the same that were in the notebook joined_data
<br>
<br>
Corruption -> Unlawful acts as defined in the United Nations Convention against Corruption and other national and international legal instruments against corruption. <br>
<br>
Corruption: Bribery ->  Promising, offering, giving, soliciting, or accepting an undue advantage to or from a public official or a person who directs or works in a private sector entity, directly or indirectly, in order that the person act or refrain from acting in the exercise of his or her official duties<br>
<br>
Corruption: Other acts of corruption ->  Other acts of corruption includes embezzlement, abuse of functions, trading in influence, illicit enrichment and all other acts of corruption not mentioned above. <br>
<br>
Fraud -> Conversion or transfer of property, knowing that such property is the proceeds of crime, for the purpose of concealing or disguising the illicit origin of such property or of helping any person who is involved in the commission of the predicate offence to evade the legal  consequences of his or her actions, as well as the concealment or disguise of the true nature, source, location, disposition, movement or ownership of rights with respect to the property.<br>
<br>
Money laundering -> Conversion or transfer of property, knowing that such property is the proceeds of crime, for the purpose of concealing or disguising the illicit origin of such property or of helping any person who is involved in the commission of the predicate offence to evade the legal consequences of his or her actions, as well as the concealment or disguise of the true nature, source, location, disposition, movement or ownership of rights with respect to the property.<br>


In [254]:
# Load and preprocess data
un_data = pd.read_excel("../data/raw/un_corruption.xlsx")
un_data.columns = un_data.iloc[1]
un_data = un_data[2:].reset_index(drop=True)

# Convert data types
un_data["year"] = pd.to_numeric(un_data["Year"])
un_data["VALUE"] = pd.to_numeric(un_data["VALUE"])

# Filter relevant data
un_data = un_data[(un_data['Indicator'] == 'Offences') &
                  (un_data['year'].between(2013, 2023)) &
                  (un_data['Unit of measurement'] == 'Rate per 100,000 population')]

# Drop unnecessary columns
un_data.drop(columns=["Dimension", "Sex", "Age", "Source", "Region", "Indicator", "Year",
                      "Unit of measurement", "Iso3_code", "Subregion"], inplace=True)

# Standardize country names
country_mapping = {
    'Netherlands (Kingdom of the)': "Netherlands",
    'United Kingdom (England and Wales)': "United Kingdom"
}
un_data["Country"] = un_data["Country"].replace(country_mapping)

# Load mapping data
mapping_data = pd.read_csv('../data/processed/merged_.csv')

# Filter out countries not in mapping data
valid_countries = set(mapping_data['country'])
un_data = un_data[un_data['Country'].isin(valid_countries)].reset_index(drop=True)

# Pivot data for better analysis
un_data = un_data.pivot(index=['Country', 'year'], columns='Category', values='VALUE').reset_index()
un_data.columns.name = None


In [255]:
un_data.sample(20)

,Country,year,Burglary,Corruption,Corruption: Bribery,Corruption: Other acts of corruption,Fraud,Fraud: Cyber-related (Cy),Money laundering,Smuggling of migrants,Theft,Theft: of a motorized vehicle,Unlawful access to a computer system,Unlawful interception or access of computer data,Unlawful interference with a computer system or computer data
326,Spain,2022,254.504811,2.169953,0.292271,1.877682,921.403759,491.881289,0.864196,0.948303,382.027409,50.554442,NaN,NaN,NaN
343,Switzerland,2019,424.682200,0.163254,0.151593,0.011661,222.736887,NaN,20.663304,0.699660,1546.785883,103.619705,7.113214,9.328805,5.072538
87,Denmark,2014,1232.370843,45.587652,0.159274,45.428378,322.776854,NaN,0.017697,5.079059,4437.947260,17.803252,3.309352,NaN,NaN
337,Switzerland,2013,849.738890,60.383017,0.173088,60.209929,119.702778,NaN,4.401383,1.594883,2405.442285,77.914368,NaN,NaN,NaN
318,Spain,2014,477.867525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.132143,78.481335,NaN,NaN,NaN
26,Belarus,2019,NaN,NaN,19.392243,NaN,NaN,NaN,NaN,0.578873,NaN,NaN,22.586382,0.206740,NaN
135,Germany,2020,358.047070,5.004261,5.004261,NaN,969.739366,280.922648,10.730960,3.726194,1118.209908,51.489885,11.964624,0.199210,4.524236
140,Greece,2015,423.767200,NaN,NaN,NaN,29.315307,8.328212,0.120296,15.768082,510.343593,245.441669,1.610121,NaN,NaN
334,Sweden,2020,782.527098,190.867560,1.465912,189.401647,2690.276855,NaN,89.719615,1.408047,2999.700067,206.114976,NaN,NaN,NaN
212,Luxembourg,2017,339.436425,0.167706,0.167706,NaN,98.610977,NaN,NaN,NaN,1692.150952,48.299254,NaN,NaN,NaN


In [256]:
un_data.drop(columns=[
    'Fraud: Cyber-related (Cy)',
    'Unlawful access to a computer system',
    'Unlawful interception or access of computer data',
    'Unlawful interference with a computer system or computer data'
], inplace=True)

## Why We’re Using (and Questioning) the UNODC Corruption Data

We're considering the **UNODC corruption and economic crime dataset** because it provides **actual reported cases**, unlike Transparency International’s CPI, which is based on perceptions. It covers specific corruption types like **bribery, fraud, and money laundering**, giving us more detailed insights.

### Pros
- **Objective Data:** Unlike CPI, it's based on real reported cases, not just opinions.
- **Comparability:** Standardized as "per 100,000 population," making cross-country comparisons easier.
- **Detailed Breakdown:** Focus on different types of corruption, which helps with targeted analysis.

### Cons
- **Underreporting Issues:** Many corruption cases likely go unreported, making data reliability questionable.
- **Legal Differences:** Definitions and enforcement levels vary across countries, affecting consistency.
- **False Comparisons:** Lower numbers don’t always mean less corruption – it could just mean weaker enforcement.


### Conclusion
It’s useful, but with limitations. We'll need to look further to get a full picture


### Data Summary

| Datapoint                                | Description                                                                                                                                                  | Data Source                                                  |
|------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------|
| Corruption Perceptions Index Rank        | Ranking based on CPI score; lower rank = better transparency.                                                                                                 | Transparency International                                  |
| Corruption Perceptions Index Score       | Score from 0 (highly corrupt) to 100 (very clean).                                                                                                           | Transparency International                                  |
| Corruption Perceptions Index Sources     | Number of sources used to calculate CPI score.                                                                                                               | Transparency International                                  |
| Corruption Perceptions Index Std. Error  | Uncertainty in CPI score; lower values = more reliability.                                                                                                   | Transparency International                                  |
| GDP per Capita (PPP)                      | Economic output per person, adjusted for purchasing power parity.                                                                                             | World Bank                                                  |
| Foreign Direct Investment (% of GDP)     | Measures foreign investment relative to GDP.                                                                                                                 | IMF, World Bank, OECD                                       |
| Unemployment Rate (% of labor force)     | Percentage of labor force without employment but seeking work.                                                                                               | Eurostat, World Bank                                        |
| Gini Index                               | Measures income inequality (0 = perfect equality, 100 = perfect inequality).                                                                                | World Bank                                                  |
| Rule of Law                              | Confidence in legal system, contract enforcement, and crime levels (-2.5 to 2.5).                                                                            | World Bank (WGI)                                            |
| Control of Corruption                    | Public power misuse for private gain (-2.5 to 2.5).                                                                                                          | World Bank (WGI)                                            |
| Government Effectiveness                 | Quality of public services and policy implementation (-2.5 to 2.5).                                                                                          | World Bank (WGI)                                            |
| Political Stability & Absence of Violence| Likelihood of political instability and violence (-2.5 to 2.5).                                                                                              | World Bank (WGI)                                            |
| Life Ladder                              | Subjective well-being rating (0 to 10).                                                                                                                      | World Happiness Report                                      |
| Social Support                           | Availability of social connections in difficult times (0 to 1).                                                                                              | World Happiness Report                                      |
| Healthy Life Expectancy                   | Expected healthy years of life at birth.                                                                                                                     | WHO, World Happiness Report                                 |
| Freedom to Make Life Choices             | Perceived personal freedom to make decisions (0 to 1).                                                                                                       | World Happiness Report                                      |
| Generosity                               | Charitable behavior and willingness to help (standardized -0.2 to 0.8).                                                                                       | World Happiness Report                                      |
| Positive Affect                          | Frequency of experiencing positive emotions (0 to 1).                                                                                                        | World Happiness Report                                      |
| Negative Affect                          | Frequency of experiencing negative emotions (0 to 1).                                                                                                        | World Happiness Report                                      |


## 3. Preprocessing the Data

Before we merge the datasets and preprocess them we will take a look at their shapes and the countries the have included.

In [257]:

datasets = {
    'cpi_data': cpi_data,
    'world_bank': world_bank,
    'happiness': happiness,
    'un_data' : un_data
}

In [258]:
import pandas as pd

def analyze_datasets(datasets):
    """
    Analyzes the shape of datasets and checks the uniqueness of the 'iso3code' column in cpi_data.

    Parameters:
        datasets (dict): A dictionary containing dataset names as keys and DataFrame objects as values.

    Returns:
        dict: A dictionary with dataset shapes and uniqueness check results.
    """

    
    iso_europe = set(countries["ISO3 Code"].dropna().astype(str).str.strip())


    for name, df in datasets.items():
        # Print dataset shape and columns
        print(f"\n{name}:")
        print(f" - Shape: {df.shape}")
        print(f" - Columns: {df.columns.tolist()}")

        # Check and process 'ISO3 Code' column
        if 'ISO3 Code' in df.columns:
            iso_codes = set(df['ISO3 Code'].dropna().astype(str).str.strip())
            missing_countries = iso_europe - iso_codes

            print(f" - Number of unique countries found: {len(iso_codes)}")
            if missing_countries:
                print(f" - Missing countries compared to reference: {missing_countries}")
            else:
                print(" - No missing countries compared to reference.")
        else:
            print(" - Column 'ISO3 Code' not found in dataset.")


# Run the analysis
analyze_datasets(datasets)





cpi_data:
 - Shape: (504, 7)
 - Columns: ['ISO3 Code', 'Economy Name', 'Year', 'Corruption Perceptions Index Rank', 'Corruption Perceptions Index Score', 'Corruption Perceptions Index Sources', 'Corruption Perceptions Index Standard Error']
 - Number of unique countries found: 42
 - No missing countries compared to reference.

world_bank:
 - Shape: (504, 10)
 - Columns: ['ISO3 Code', 'Year', 'Rule_of_law', 'Gov_effectiveness', 'Control_of_corruption', 'Fdi', 'Gdp_per_capita', 'Gini', 'Unemployement', 'Political_stability']
 - Number of unique countries found: 42
 - No missing countries compared to reference.

happiness:
 - Shape: (469, 10)
 - Columns: ['Country name', 'Year', 'Life Ladder', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Generosity', 'Positive affect', 'Negative affect', 'ISO3 Code']
 - Number of unique countries found: 40
 - Missing countries compared to reference: {'TUR', 'CZE'}

un_data:
 - Shape: (360, 11)
 - Columns: ['Count

### 3.1 Merge The Data

Here we load all explored and preprocessed datasets, to combine them into one large dataset. We also standardize the country_code and year column, as we use it to join the datasets.

In [259]:
happiness.rename(columns={ 'Year': 'year'}, inplace=True)
cpi_data.rename(columns={ 'Year': 'year'}, inplace=True)
world_bank.rename(columns={'Year': 'year'}, inplace=True)
un_data.rename(columns={'Year': 'year'}, inplace=True)

In [260]:
merged_df = None

for name, df in datasets.items():
    # Ensure 'ISO3 Code' and 'year' are in all dataframes
    if 'ISO3 Code' in df.columns and 'year' in df.columns:
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=['ISO3 Code', 'year'], how='outer')
    else:
        print(f"Skipping {name}: required columns missing.")

# Fill NaN values with placeholder if necessary
merged_df.fillna('N/A', inplace=True)

# Display the final merged dataset
print("\nMerged Dataset:")
print(merged_df)

ValueError: You are trying to merge on int64 and object columns for key 'year'. If you wish to proceed you should use pd.concat

In [221]:
merged_df.head()

,ISO3 Code,country,year,Corruption Perceptions Index Rank,Corruption Perceptions Index Score,Corruption Perceptions Index Sources,Corruption Perceptions Index Standard Error,Country name,Life Ladder,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Positive affect,Negative affect
0,ALB,Albania,2012,113.0,33.0,7.0,2.0,Albania,5.510124,0.784502,68.160004,0.601512,-0.172988,0.553473,0.271393
1,ALB,Albania,2013,116.0,31.0,7.0,2.1,Albania,4.550648,0.759477,68.440002,0.63183,-0.131339,0.540751,0.338379
2,ALB,Albania,2014,110.0,33.0,7.0,1.51,Albania,4.813763,0.625587,68.720001,0.734648,-0.028801,0.572945,0.334543
3,ALB,Albania,2015,88.0,36.0,7.0,3.58,Albania,4.606651,0.639356,69.0,0.703851,-0.084982,0.579072,0.350427
4,ALB,Albania,2016,83.0,39.0,7.0,1.99,Albania,4.511101,0.638411,69.025002,0.729819,-0.021136,0.56708,0.321706


## 4. Explore the Data


## 5. Model the Data


## 6. Communicate and Visualize the Results / Is the Question answered? Why?

* What questions did you ask of the data? 
* Why were these good questions? 
* What were the answers to these questions? 
* How did you obtain them? 
* Do the answers make sense? 
* Were there any difficulties in analysing the data? 
* What were the key insights obtained? 
* What are potential biases in the data and analysis? 
* hich Data Science tools and techniques were learned during this exercise?

• This notebook should be accompanied by a 2-page PDF document that presents a summary
of the main insights into the data obtained — this is a management summary, so
should be written in a way that is easy to understand by managers. It should also justify
why the insights obtained make sense — include diagrams. Do not try and summarise everything
that you did in the two pages – focus on the insights. Only the first two pages of
the submitted document will be read — do not add a cover page.
• Data needed by the Jupyter notebook should either be accessed directly at its source in the
code, or included in the zip file (in the sub-directories expected in the Python code). If some
of the data is too large to include in the zip file and cannot be accessed directly within the
code, then include a file named install data.txt that includes full instructions on where
to download the data and in which sub-directories to install it so that the Python code in
the Jupyter notebook can execute.
There are various online tools for collaborating on Jupyter notebooks. One free possibility is
Kaggle Notebooks: https://www.kaggle.com/docs/notebooks
Note that 36 hours per person is foreseen for this exercise, which is around half the time
foreseen for the course (75 hours). This means that everyone should work for around a standard
working week on this exercise, so four weeks effort for a group of four. The evaluation will be based
on the expectation of a manager assigning such a task to a group of four junior data scientists for
a week. Note that this expectation is not met by submitting an overly long Jupyter notebook —
you need to demonstrate that:
• You have approached the analysis in a logical and structured way.
• You have learned some new data science tools and techniques.
• You have gained new insights into the data.

Overly long notebooks with little substance will be penalised.
Use any additional information that you wish — document which information you use in the
Jupyter notebook. If you use Large Language Models (LLMs) then add a section in the Jupyter
Notebook documenting exactly what LLMs were used for and how they were used. Releasing your
Notebook as a public Kaggle Notebook will be well received.